In [ ]:
###   Imports and Logging   ###
# Default Python stuff
import sys
import logging
import math
import copy
from dataclasses import dataclass, field, replace
# SciPy stuff
import numpy as np
import pandas as pd
import scipy.optimize
import matplotlib.pyplot as plt
from matplotlib import __version__ as pltver
import matplotlib.ticker as ticker
# Sci-Kit Aero stuff
from skaero.atmosphere import coesa
from skaero import __version__ as skver

# Logging setup
logging.basicConfig(
    level=logging.INFO,
    format=' %(asctime)s -  %(levelname)s -  %(message)s'
)
# Spit out the runtime info for reproducibility
logging.info('=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=')
logging.info('Python version: {}'.format(sys.version))
logging.info('Author: Benjamin Crews')
logging.info('Numpy version: {}'.format(np.version.version))
logging.info('Pandas version: {}'.format(pd.__version__))
logging.info('Matplotlib version: {}'.format(pltver))
logging.info('SciKit-Aero version: {}'.format(skver))
logging.info('=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=')

In [55]:
### Classes

# This cell contains the basic class definition. One the most important classes will be the Helicopter class.
# This class is the whole purpose of the 'OOP-Conversion' branch, to transition the code to an Object-Oriented
# Philosophy, which will drastically decrese complexity for funtion calls and scale the capability.
@dataclass
class Helicopter():
    '''
    This class represents a helicopter with typical design features.
    These features are:
       Single Main Rotor,
       Single Tail Rotor,
       No shared lift or forward thrusters
       
    The basic Helicopter class has attributes and properties (according to Python definitions).
    Defaults are set for all values, so be careful with results before checking that all your
    input values (and units) are correct!
    
    This class uses a constant equivalent chord for all blades, and an efficiency factor
    must be used to estimate real performance based on this.
    '''
    ## Utility Attributes
    name: str = field(default='Tim the Enchanter')
    rotors: tuple = ('MR', 'TR')
    
    ## Main Rotor Data
    MR_dia: float = field(default=10, metadata={'units':'ft'})
    MR_b: int = field(default=2, metadata={'units':''})
    MR_ce: float = field(default=10.4, metadata={'units':'in'})
    MR_Omega: float = field(default=43.2, metadata={'units':'rad/s'})
    MR_cd0: float = field(default=0.0080, metadata={'units':''})
    
    ## Tail Rotor Data
    TR_dia: float = field(default=2, metadata={'units':'ft'})
    TR_b: int = field(default=2, metadata={'units':''})
    TR_ce: float = field(default=6, metadata={'units':'in'})
    TR_Omega: float = field(default=20, metadata={'units':'rad/s'})
    TR_cd0: float = field(default=0.015, metadata={'units':''})
    
    ## Airframe Data
    GW_empty: float = field(default=1000, metadata={'units':'lbs'})
    GW_fuel: float = field(default=0, metadata={'units':'lbs'})
    download: float = field(default=0.03, metadata={'units':'%'})
    HIGE_factor: float = field(default=1.2)
    # Equivalent flat-plate-drag area
    fe: float = field(default=5, metadata={'units':'ft2'})
    # Tail length to TR center of thrust
    l_tail: float = field(default=15, metadata={'units':'ft'})
    # Vertical Tail Area
    S_vt: float = field(default=15, metadata={'units':'ft2'})
    # Coefficient of Lift for the vertical tail
    cl_vt: float = field(default=0.1, metadata={'units':''})
    # Vertical tail aspect ratio
    AR_vt: float = field(default=3, metadata={'units':''})
        
    ## Engine Data
    eta_MRxsmn: float = field(default=0.985, metadata={'units':'%'})
    eta_TRxsmn: float = field(default=0.9712, metadata={'units':'%'})
    eta_xsmn_co: float = field(default=0.986, metadata={'units':'%'})
    pwr_acc: float = field(default=10, metadata={'units':'hp'})
    eta_inst: float = field(default=0.95, metadata={'units':'%'})
    xsmn_lim: float = field(default=674, metadata={'units':'hp'})
    pwr_lim: float = field(default=813, metadata={'units':'hp'})

    
    def __post_init__(self):
        '''
        This method automatically calculates the extra input characteristics,
        which are based on the defined values.
        '''
        self.MR_R = self.MR_dia/2
        self.MR_A = math.pi * self.MR_R**2
        self.MR_vtip = self.MR_Omega * self.MR_R
        self.MR_sol = self.MR_b*self.MR_ce/(12*math.pi*self.MR_R)
        
        self.TR_R = self.TR_dia/2
        self.TR_A = math.pi * self.TR_R**2
        self.TR_vtip = self.TR_Omega * self.TR_R
        self.TR_sol = self.TR_b*self.TR_ce/(12*math.pi*self.TR_R)
        
    
    def __str__(self) -> str:
        '''
        Human-friendly representation of the helicopter class
        '''
        out = '-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-\n'
        out += f'{self.name:^45}'
        out += f'\nRotors: {self.rotors}'
        out += '\n-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-'
        out += '\nMain Rotor Inputs:'
        for k,v in self.__dict__.items():
            if k.startswith('MR'):
                out += '\n{:>17}: {:>7.3f} [{}]'.format(k, v, self.get_units(k))
                
        out += '\n-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-'
        out += '\nTail Rotor Inputs:'
        for k,v in self.__dict__.items():
            if k.startswith('TR'):
                out += '\n{:>17}: {:>7.3f} [{}]'.format(k, v, self.get_units(k))
                
        out += '\n-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-'
        out += '\nAirframe Data:'
        for k in ['GW_empty', 'download', 'HIGE_factor', 'fe', 'l_tail', 'S_vt', 'cl_vt', 'AR_vt']:
            out += '\n{:>17}: {:>7.3f} [{}]'.format(k, self.__dict__[k], self.get_units(k))
            
        out += '\n-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-'
        out += '\nEngine Data:'
        for k in ['eta_MRxsmn', 'eta_TRxsmn', 'eta_xsmn_co', 'eta_inst', 'xsmn_lim', 'pwr_lim']:
            out += '\n{:>17}: {:>7.3f} [{}]'.format(k, self.__dict__[k], self.get_units(k))
        
        out += '\n-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-'
        return out
                
    
    def get_units(self, attribute_name) -> str:
        '''Returns the units from the metadata definition of the attribute.'''
        try:
            return self.__dataclass_fields__[attribute_name].metadata['units']
        except KeyError:
            return ''
        
    @property
    def GW(self):
        return self.GW_empty + self.GW_fuel
    
    
    def HOGE(self,
             atm,
             Thrust = None,
             delta_1: float = -0.0216,
             delta_2: float = 0.4,
             k_i: float = 1.1,
             Vroc: float = 0
            ) -> list:
        '''
        This method calculates Hover Out of Ground Effect performance.
        
        Inputs:
            atm is an Environment class object, which provides altitude and temperature.
            delta_1 is the second term in the 3-part drag equation (default to -0.0216 based on literature)
            delta_2 is the third term in the 3-part drag equation (default to 0.4 based on literature)
            k_i is the "efficiency factor" which includes losses for non-uniform inflow, and non-ideal twist.
            Vroc is the vertical rate of climb, in ft/min.
            
        Returns a list of metrics.
                  a = 3D lift coefficient [cl/rad]
            delta_0 = corrected, compressible drag coefficient (1st term in 3-term drag equation)
                 Ct = coefficient of thrust
               Cq_i = coefficient of torque, induced velocity contribution
               Cq_v = coefficient of torque, vroc contribution
               Cq_0 = coefficient of torque, 1st term drag
               Cq_1 = coefficient of torque, 2nd term drag
               Cq_2 = coefficient of torque, 3rd term drag
                  Q = Main Rotor Torque
               P_MR = Main Rotor required Power
               P_TR = Tail Rotor required Power
            SHP_ins = Total shaft horsepower of the installed engine
          SHP_unins = Total shaft horsepower of an uninstalled engine (spec)
        '''
        # If thrust is not specified (default),
        # Set it to weight plus download
        if not Thrust:
            Thrust = self.GW*(1+self.download)
        
        Vroc = Vroc/60   # Convert the climb rate into ft/s for calculations.
        
        ###   Airfoil Lift factor correction from 2d to 3d   ###
        AR = 12*self.MR_R/self.MR_ce
        a_0 = 2*math.pi
        a = a_0 / (1 + a_0/(math.pi*AR))
        
        ###   Compressibility Correction Factor   ###
        c_sound = math.sqrt(1.4*1716.4*atm.T)
        mach_08 = self.MR_vtip*0.8/c_sound
        delta_0 = self.MR_cd0/math.sqrt(abs(mach_08**2 - 1))
        
        Ct = Thrust/(atm.rho*self.MR_A*self.MR_vtip**2)
        
        # Get B correction for tip-loss = 1 - (sqrt(2*Ct)/b)
        B = 1 - math.sqrt(2*Ct)/self.MR_b
        
        # MR Torque/Power Calculations
        Cq_i = 0.5*Ct*math.sqrt( (Vroc/(self.MR_Omega*self.MR_R))**2 + 2*Ct/B**2 )
        Cq_v = Vroc*Ct/(2*self.MR_Omega*self.MR_R)
        Cq_0 = self.MR_sol*delta_0/8
        Cq_1 = (2*delta_1/(3*a))*(Ct/B**2)
        Cq_2 = (4*delta_2/(self.MR_sol*a**2))*(Ct/B**2)**2

        Cq = Cq_i + Cq_v + Cq_0 + Cq_1 + Cq_2

        P_MR = Cq*atm.rho*self.MR_A*self.MR_vtip**3   # [ft*lbs/s]
        HP_MR = P_MR*k_i/550
        Q = HP_MR*550*self.MR_R/self.MR_vtip    # [lb*ft]
        
        # TR Torque/Power Calculations
        T_tr = Q/(2*self.l_tail)      # [lbs]
        Ct_tr = T_tr/(atm.rho*self.TR_A*self.TR_vtip**2)
        B_tr = 1 - math.sqrt(2*Ct_tr)/self.TR_b
        vi_tr = math.sqrt(T_tr/(2*atm.rho*math.pi*(B_tr*self.TR_R)**2))   # [ft/s]
        # Induced horsepower
        HPi_tr = T_tr*vi_tr/550  # [hp]
        # Profile horsepower
        HPpro_tr = self.TR_sol*self.TR_cd0*atm.rho*math.pi*self.TR_R**2*self.TR_vtip**3 / 4400   # [hp]
        HP_TR = HPi_tr + HPpro_tr   # [hp]
        
        # Combine all the required power
        SHP_ins = HP_MR + HP_TR + self.pwr_acc +                  \
                  HP_MR*(1-(self.eta_MRxsmn*self.eta_xsmn_co)) + \
                  HP_TR*(1-self.eta_TRxsmn*self.eta_xsmn_co)   + \
                  self.pwr_acc*(self.eta_xsmn_co)
        SHP_unins = SHP_ins/self.eta_inst   # [hp]
        
        # Warn the user if limits have been exceeded.
        if SHP_unins > self.pwr_lim:
            logging.warning('Engine Power required to hover is greater than Engine rated limit!')
        elif SHP_unins > self.xsmn_lim:
            logging.warning('Engine Power required to hover is greater than gearbox capability!')
        
        return {'a':a, 'delta_0':delta_0, 'Ct':Ct,
                'Cq_i':Cq_i, 'Cq_v':Cq_v, 'Cq_0':Cq_0, 'Cq_1':Cq_1, 'Cq_2':Cq_2, 'Cq':Cq,
                'Q':Q, 'P_MR':P_MR, 'HP_MR':HP_MR,'HP_TR':HP_TR,
                'SHP_ins':SHP_ins, 'SHP_unins':SHP_unins}
    
    
    def HIGE(self,
             atm,
             Thrust = None,
             delta_1: float = -0.0216,
             delta_2: float = 0.4,
             k_i: float = 1.1,
             Vroc: float = 0
            ) -> list:
        '''
        This method calculates the Hover In Ground Effect performance.
        
        This is simply the HOGE, but with a factored thrust.
        '''
        if not Thrust:
            Thrust = self.GW*(1+self.download)/self.HIGE_factor
        else:
            Thrust = Thrust/self.HIGE_factor
            
        return self.HOGE(atm, Thrust, delta_1, delta_2, k_i, Vroc)
    
    
    def forward_flight(self,
                       atm,
                       Airspeed
                      ) -> dict:
        '''
        This function evaluates performance in forward flight.
        Airspeed (in kts) input can be a single value, or a list of the desired speed sweep.
        
        Performance metrics such as drag, MR power, TR power, Engine power, fuel
        consumption, and range are evaluated.
        
        A dictionary is returned with keys for each characteristic and
        a list of outputs as values.
        '''
        # Convert airspeed input to a list of a signle value, otherwise, pass it on.
        Airspeed = Airspeed if type(Airspeed) is list else [Airspeed]
            
        df = pd.DataFrame(data=Airspeed, columns=['Airspeed'])
        
        ## Main Rotor Calculations
        # Dynamic Pressure = 1/2 rho V^2
        df['q'] = 0.5*atm.rho*(df.Airspeed * 1.68781)**2    # 1.689 is simply the conversion factor from kts to ft/s
        # Advance Ratio
        df['mu'] = df.Airspeed * 1.689 / self.MR_vtip
        # Hover Induced Velocity
        Thrust = self.GW*(1+self.download)
        Ct = Thrust/(atm.rho*self.MR_A*self.MR_vtip**2)
        B = 1 - math.sqrt(2*Ct)/self.MR_b
        v_0 = math.sqrt(Thrust/(2*atm.rho*math.pi*(B*self.MR_R)**2))
        # Induced velocity in Forward Flight, using Glauert's Model
        df['v_if'] = v_0 * (-0.5*(df.Airspeed/v_0)**2 + np.sqrt((df.Airspeed/v_0)**4 / 4 + 1))
        # Thrust coef. over solidity
        df['Cts'] = Thrust/(atm.rho*self.MR_A*self.MR_vtip**2) / self.MR_sol
        # Blade loading
        df['tc'] = 2*df.Cts
        # Empirical lower bound of blade loading
        df['tc_lower'] = -0.6885*df.Cts + 0.3555
        # Change in Drag Coef. (due to retreating blade stall)
        # From a NASA CR
        df['F'] = ((df.Cts/(1-df.mu)**2) * (1 + self.fe*df.q/self.GW)) - 0.1376
        df['del_cds'] = 18.3*(1-df.mu)**2*df.F**3
        # This value should never be less than zero, so clip it.
        df.del_cds.clip(lower=0, inplace=True)
        # Mach Drag Divergence of the airfoil: At what Mach would shockwaves start to form?
        df['MDD'] = 0.82 - 2.4*df.Cts
        # MY90. Mach at the tip
        c_sound = math.sqrt(1.4*1716.4*atm.T)
        df['MY90'] = (df.Airspeed*1.68781 + self.MR_vtip)/c_sound
        # Change in Drag Coef. from compressibility
        df['del_cdcomp'] = 0.2*(df.MY90 - df.MDD)**3 + 0.0085*(df.MY90 - df.MDD)
        # Total Drag Coef.
        df['cd'] = 0.0095 + df.del_cds + df.del_cdcomp
        # Induced Horsepower
        df['Hp_ind'] = Thrust*df.v_if/550
        # Profile Horsepower
        df['Hp_pro'] = self.MR_sol*df.cd*(1+4.65*df.mu**2)*atm.rho*math.pi*self.MR_R**2*self.MR_vtip**3/4400
        # Parasite Power
        df['Hp_par'] = self.fe*atm.rho*(df.Airspeed*1.68781)**3/1100
        # Main Rotor Horsepower
        df['MR_hp'] = df.Hp_ind + df.Hp_pro + df.Hp_par
        # Main Rotor Torque
        df['MR_Q'] = 5252*df.MR_hp/(self.MR_Omega*60/(2*math.pi))
        
        ## Tail Rotor Calculations
        # Anti-torque Required from Tail Rotor Thrust
        # MR Torque over the moment arm
        df['T_at'] = df.MR_Q/(2*self.l_tail)
        
        return df
    
    

@dataclass
class Environment():
    '''
    This class contains all the atmospheric data used in performance calculations.
    All atmospheric properties are attributes of this class.
    
    Depends on sk-aero.coesa module.
    Note that only input is the altitude, in feet. All units are automatically
    converted from metric to Imperial.
    '''
    alt: float = field(default=0, metadata={'units':'ft'})
    
    def __post_init__(self):
        # an atmosphere object at a height
        self.atm = coesa.table(self.alt/3.28084)
        self.T = self.atm[1]*1.8         # [Rankine]
        self.p = self.atm[2]/6895        # [psi]
        self.rho = self.atm[3]/515       # [slug/ft3]
        
    # TODO: def __str__()

In [56]:
## Build the Project Helicopter
heli = Helicopter(name='Project Helicopter Spec',
                  MR_dia = 35,
                    MR_b = 4,
                   MR_ce = 10.4,
                MR_Omega = 43.2,
                  MR_cd0 = 0.0080,
                  TR_dia = 5.42,
                    TR_b = 4,
                   TR_ce = 7,
                TR_Omega = 239.85,
                  TR_cd0 = 0.015,
                GW_empty = 5000,
                download = 0.03,
                      fe = 12.9,
                  l_tail = 21.21,
                    S_vt = 20.92,
                   cl_vt = 0.22,
                  AR_vt = 3
                 )
print(heli)

-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-
           Project Helicopter Spec           
Rotors: ('MR', 'TR')
-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-
Main Rotor Inputs:
           MR_dia:  35.000 [ft]
             MR_b:   4.000 []
            MR_ce:  10.400 [in]
         MR_Omega:  43.200 [rad/s]
           MR_cd0:   0.008 []
             MR_R:  17.500 []
             MR_A: 962.113 []
          MR_vtip: 756.000 []
           MR_sol:   0.063 []
-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-
Tail Rotor Inputs:
           TR_dia:   5.420 [ft]
             TR_b:   4.000 []
            TR_ce:   7.000 [in]
         TR_Omega: 239.850 [rad/s]
           TR_cd0:   0.015 []
             TR_R:   2.710 []
             TR_A:  23.072 []
          TR_vtip: 649.993 []
           TR_sol:   0.274 []
-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-
Airframe Data:
         GW_empty: 5000.000 [lbs]
         download:   0.030 [%]
      HIGE_factor:   1.200 []
               fe:  12.900 [ft2]
          

In [57]:
atm = Environment(0)
print('-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-')
print('{:^45}'.format('Results - HOGE'))
print('-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-')
for k,v in heli.HOGE(atm).items():
    print('{:>17}:  {:>7.3e}'.format(k, v))
    
print('-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-')

print('\n\n-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-')
print('{:^45}'.format('Results - HIGE'))
print('-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-')
for k,v in heli.HIGE(atm).items():
    print('{:>17}:  {:>7.3e}'.format(k, v))
    
print('-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-')


-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-
               Results - HOGE                
-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-
                a:  5.717e+00
          delta_0:  9.518e-03
               Ct:  3.937e-03
             Cq_i:  1.787e-04
             Cq_v:  0.000e+00
             Cq_0:  7.502e-05
             Cq_1:  -1.037e-05
             Cq_2:  1.317e-05
               Cq:  2.565e-04
                Q:  6.458e+03
             P_MR:  2.536e+05
            HP_MR:  5.072e+02
            HP_TR:  2.469e+01
          SHP_ins:  5.674e+02
        SHP_unins:  5.973e+02
-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-


-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-
               Results - HIGE                
-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-
                a:  5.717e+00
          delta_0:  9.518e-03
               Ct:  3.281e-03
             Cq_i:  1.356e-04
             Cq_v:  0.000e+00
             Cq_0:  7.502e-05
             Cq_1:  -8.610e-06
             C

In [ ]:
#######################################
###   Forward Flight Calculations   ###
#######################################
logging.info('Beginning Forward Flight Calculations...')


    #    Main Rotor Calculations
    ##################################

# Max Airspeed, in knots
VNE = 160
# Equivalent Flat-Plate Drag Area
fe = 12.4   # [ft2]


df = pd.DataFrame(data=[20], columns=['Airspeed'])

# Dynamic Pressure = 1/2 rho V^2
df['q'] = 0.5*rho*(df.Airspeed * 1.68781)**2    # 1.689 is simply the conversion factor from kts to ft/s

# Advance Ratio
df['mu'] = df.Airspeed * 1.689 / v_tip

# Hover Induced Velocity
v_0 = math.sqrt(THRUST/(2*rho*math.pi*(B*R)**2))
# Induced velocity in Forward Flight, using Glauert's Model
df['v_if'] = v_0 * (-0.5*(df.Airspeed/v_0)**2 + np.sqrt((df.Airspeed/v_0)**4 / 4 + 1))

# Thrust coef. over solidity
df['Cts'] = THRUST/(rho*A*v_tip**2) / sol

# Blade loading
df['tc'] = 2*df.Cts

# Empirical lower bound of blade loading
df['tc_lower'] = -0.6885*df.Cts + 0.3555

# Change in Drag Coef. (due to retreating blade stall)
# From a NASA CR
df['F'] = ((df.Cts/(1-df.mu)**2) * (1 + fe*df.q/GW)) - 0.1376
df['del_cds'] = 18.3*(1-df.mu)**2*df.F**3
# This value should never be less than zero, so clip it.
df.del_cds.clip(lower=0, inplace=True)

# Mach Drag Divergence of the airfoil: At what Mach would shockwaves start to form?
df['MDD'] = 0.82 - 2.4*df.Cts

# MY90. Mach at the tip
df['MY90'] = (df.Airspeed*1.68781 + v_tip)/c_sound

# Change in Drag Coef. from compressibility
df['del_cdcomp'] = 0.2*(df.MY90 - df.MDD)**3 + 0.0085*(df.MY90 - df.MDD)

# Total Drag Coef.
df['cd'] = 0.0095 + df.del_cds + df.del_cdcomp

# Induced Horsepower
df['Hp_ind'] = THRUST*df.v_if/550

# Profile Horsepower
df['Hp_pro'] = sol*df.cd*(1+4.65*df.mu**2)*rho*math.pi*R**2*v_tip**3/4400

# Parasite Power
df['Hp_par'] = fe*rho*(df.Airspeed*1.68781)**3/1100

# Main Rotor Horsepower
df['MR_hp'] = df.Hp_ind + df.Hp_pro + df.Hp_par

# Main Rotor Torque
df['MR_Q'] = 5252*df.MR_hp/(OMEGA*60/(2*math.pi))

# Anti-torque Required from Tail Rotor Thrust
# MR Torque over the moment arm
df['T_at'] = df.MR_Q/(2*l_tail)

MR = df.copy()
MR

In [ ]:

    #    Tail Rotor Calculations
    #################################

# Copy the dataframe to a new variable, for Tail calculations
# Carry over the airspeed and anti-torque thrust
TR = pd.DataFrame(data=MR.Airspeed)
TR['T_at'] = MR.T_at.copy()
TR['q'] = MR.q.copy()

# Calculate the anti-torque provided
# by the vertical tail
# Lift = cl*wing_area*dynamic_pressure
TR['L_vt'] = cl_vt*S_vt*TR.q

# Anti-torque minus Vfin Lift
TR['TTR'] = TR.T_at - TR.L_vt

# Induced Drag from the Vfin
TR['D_vt'] = TR.L_vt**2/(2*TR.q*S_vt*AR_vt)

# Calculate the "hover induced velocity" of TR
TR['v0_tr'] = np.sqrt(abs(TR.TTR)/(2*rho*math.pi*R_tr**2))

# Calculate the forward flight induced velocity
TR['v_if_tr'] = TR.v0_tr * (-0.5*(TR.Airspeed/TR.v0_tr)**2 + np.sqrt((df.Airspeed/TR.v0_tr)**4 / 4 + 1))

# Induced Horsepower
TR['HP_i_tr'] = TR.TTR*TR.v_if_tr/550

# Profile Horsepower
TR['HP_pro_tr'] = sol_tr*cd0_tr*(1+4.65*MR.mu**2)*rho*math.pi*R_tr**2*v_tip_tr**3 / 4400

# Total TR Horsepower
TR['HP_tr'] = TR.HP_i_tr + TR.HP_pro_tr

TR

In [ ]:

    #   Power Calculations
    #############################

Power = pd.DataFrame(data=MR.Airspeed)
Power['MR_hp'] = MR.MR_hp.copy()
Power['TR_hp'] = TR.HP_tr.copy()

Power['del_MRxsmn'] = Power.MR_hp*(1-(eta_MRxsmn*eta_xsmn_co))
Power['del_TRxsmn'] = Power.TR_hp*(1-eta_TRxsmn)
Power['del_Acc_co'] = pwr_acc*(1-eta_xsmn_co)

Power['SHP_inst_req'] = Power.MR_hp + Power.TR_hp + Power.del_MRxsmn \
                      + Power.del_TRxsmn + pwr_acc + Power.del_Acc_co

Power['del_inst'] = Power.SHP_inst_req*(1-eta_inst)
Power['SHP_uninst'] = Power.SHP_inst_req + Power.del_inst

Power['L_D'] = GW*Power.Airspeed*1.689 / (550*Power.SHP_uninst)

Power['Pwr_ratio'] = Power.SHP_uninst/pwr_lim

# TODO: Calculate fuel consumption based on power
#       I have no idea how to do this...
# Power['sfc'] = 

Power

In [ ]:
## This cell just plots the Speed-Power curve

fig, ax = plt.subplots(figsize=(15,9))

# Add the data and color it
ax.plot(Power.Airspeed, Power.SHP_inst_req, color='orange', label='Installed Power', marker='o', markersize='4')
ax.plot(Power.Airspeed, Power.SHP_uninst, color='green', label='Uninstalled Power', marker='o', markersize='4')
ax.legend()

# Axis labels
ax.set_xlabel('Airspeed, $V$ [kts]', fontsize=12)
ax.set_ylabel('Engine Power, $P$ [hp]', fontsize=12)
ax.set_title('Speed-Power Polar\n', fontsize=18)

# Move the axis and its label to the top
# ax.xaxis.set_label_position('top')
# ax.xaxis.tick_top()

# Set the ticks
# ax.xaxis.set_major_locator(ticker.FixedLocator([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]))
ax.tick_params(which='minor', width=0.75, length=2.5)
ax.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax.yaxis.set_minor_locator(ticker.AutoMinorLocator())
ax.tick_params(axis='both', which='both', direction='in')

# Set the grid lines
ax.grid(b=True, which='major', linestyle=':')
ax.grid(b=True, which='minor', linestyle=':', alpha=0.3)